# Segmenting and Clustering Neighborhoods in Toronto

### Lybraries

In [1]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
import json # library to handle JSON files

from bs4 import BeautifulSoup
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

import bs4 as bs
import urllib.request
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# geocoder
#!conda install -c conda-forge geocoder --yes #Uncomment this row to install geocoder in case you don't have it
import geocoder

#!conda install -c conda-forge geopy --yes # #Uncomment this row to install geocoder in case you don't have it
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes #Uncomment this row to install geocoder in case you don't have it
import folium # map rendering library

print('Libraries imported.')

/Users/mauroferri/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


Libraries imported.


### Building the dataframe for the Toronto Neighbourhoods using BeautifulSoup.


In [2]:
# Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
source = urllib.request.urlopen(url).read()
soup = bs.BeautifulSoup(source,'lxml')


table = soup.find('table')
table_rows = table.find_all('tr')


line = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        line.append(row)
        
df = pd.DataFrame(line, columns=["PostalCode", "Borough", "Neighbourhood"])
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,None
1,M2A,Not assigned,None
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,None
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


In [3]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned']
df.head(10)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [4]:
# More than one neighborhood can exist in one postal code area.
# Wikipedia manage this case, using as a separator for Neighbourhood "/"
# Replace "/" with ","

df['Neighbourhood'] = df['Neighbourhood'].str.replace('/',',')
df.head(10)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern , Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill , Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
# There is not any Neighboorhodd with 'Not assignesd' value'
count = df[df.Neighbourhood == 'Not assigned'].count()
count

PostalCode       0
Borough          0
Neighbourhood    0
dtype: int64

In [6]:
# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
df.shape

(103, 3)

### Project Part 2: Getting the latitude and the longitude coordinates of each neighborhood.


In [7]:
# geocoder, at the present, doesn't work, so I'm going to use csv file

#df['Latitude'] = ''
#df['Longitude'] = ''

# loop through the postal code get latitude and longitude
#for index, data in df.iterrows():
#    lat_lng_coords = None
#    while(lat_lng_coords is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format(data['PostalCode']))
#        lat_lng_coords = g.latlng
#    data['Latitude'] = lat_lng_coords[0]
#    data['Longitude'] = lat_lng_coords[1]
#    print ('PostalCode:', data['PostalCode'], 'Latitude:', data['Latitude'], 'Longitude:', data['Longitude'])


In [8]:
csv_ll = pd.read_csv('http://cocl.us/Geospatial_data')

In [9]:
csv_ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
csv_ll.rename(columns={'Postal Code': 'PostalCode'}, inplace = True)
df = pd.merge(df, csv_ll, on ='PostalCode')

In [11]:
df.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


### Project Part 3: Segmenting and Clustering the neighbourhoods in Toronto.

1. For this project, I'll focus on the Borough of Central Toronto.

2. K-Clustering will be used to find out the most common venues and then segment and group them accordingly.

3. Only the Top 5 clusters will be displayed.

In [12]:
#Use geopy to get the latitude and longitude of Toronto
address = 'Toronto Ontario, TO'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Ontario are {}, {}.'.format(latitude, longitude))

/Users/mauroferri/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto Ontario are 43.6534817, -79.3839347.


In [13]:
# I use Folium library to create map of Toronto using latitude and longitude
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [45]:
# Filter by Central Toronto and create new DataFrame
CT_data = df[df['Borough'] == 'Central Toronto'].reset_index(drop=True)
CT_data.head(10)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
5,M5R,Central Toronto,"The Annex , North Midtown , Yorkville",43.672710,-79.405678
6,M4S,Central Toronto,Davisville,43.704324,-79.388790
7,M4T,Central Toronto,"Moore Park , Summerhill East",43.689574,-79.383160
8,M4V,Central Toronto,"Summerhill West , Rathnelly , South Hill , For...",43.686412,-79.400049


In [15]:
address = 'Central Toronto, Toronto Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

/Users/mauroferri/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Central Toronto are 43.6449033, -79.3818364.


In [16]:
# create map of Central Toronto
map_CT = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(CT_data['Latitude'], CT_data['Longitude'], CT_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_CT)  
    
map_CT

In [17]:
# Foursquare Credentials
CLIENT_ID = 'AU22VEFS5ZUGLZMJCKXLYMWEQZYYUMPGP03PTEA440LFZWD2' # your Foursquare ID
CLIENT_SECRET = 'VBZL450SSY15H2ZTKX3LN45IJRLFKE1QCFWTO1KZTQ3KCADW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: AU22VEFS5ZUGLZMJCKXLYMWEQZYYUMPGP03PTEA440LFZWD2
CLIENT_SECRET:VBZL450SSY15H2ZTKX3LN45IJRLFKE1QCFWTO1KZTQ3KCADW


In [18]:
# explore the first neighborhood
CT_data.loc[0, 'Neighbourhood']

'Lawrence Park'

In [19]:
neighborhood_latitude = CT_data.loc[0, 'Latitude'] # Lawrence Park latitude value
neighborhood_longitude = CT_data.loc[0, 'Longitude'] # Lawrence Park longitude value

neighborhood_name = CT_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


In [20]:
# explore the top 100 venues
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AU22VEFS5ZUGLZMJCKXLYMWEQZYYUMPGP03PTEA440LFZWD2&client_secret=VBZL450SSY15H2ZTKX3LN45IJRLFKE1QCFWTO1KZTQ3KCADW&v=20180605&ll=43.7280205,-79.3887901&radius=500&limit=100'

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ead158f29ce6a001bb26ec2'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.7325205045, 'lng': -79.3825744605273},
   'sw': {'lat': 43.7235204955, 'lng': -79.3950057394727}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.72696303913755,
          'lng': -79.39438246708775}],
        'distance': 465,
        'c

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
# clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/Users/mauroferri/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Averax Group,Construction & Landscaping,43.727406,-79.383103
2,Zodiac Swim School,Swim School,43.728532,-79.382860
3,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [25]:
# Explore Neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
# get Central Toronto venues
CT_venues = getNearbyVenues(names=CT_data['Neighbourhood'],
                                   latitudes=CT_data['Latitude'],
                                   longitudes=CT_data['Longitude']
                                  )

print(CT_venues.shape)
CT_venues.head(10)

Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
North Toronto West
The Annex , North Midtown , Yorkville
Davisville
Moore Park , Summerhill East
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
(114, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Averax Group,43.727406,-79.383103,Construction & Landscaping
2,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
3,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
4,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden
5,Davisville North,43.712751,-79.390197,Homeway Restaurant & Brunch,43.712641,-79.391557,Breakfast Spot
6,Davisville North,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park
7,Davisville North,43.712751,-79.390197,Summerhill Market North,43.715499,-79.392881,Food & Drink Shop
8,Davisville North,43.712751,-79.390197,Winners,43.713236,-79.393873,Department Store
9,Davisville North,43.712751,-79.390197,Best Western Roehampton Hotel & Suites,43.708878,-79.390880,Hotel


In [27]:
#explore unique categories
CT_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Davisville,36,36,36,36,36,36
Davisville North,8,8,8,8,8,8
Forest Hill North & West,5,5,5,5,5,5
Lawrence Park,4,4,4,4,4,4
"Moore Park , Summerhill East",1,1,1,1,1,1
North Toronto West,20,20,20,20,20,20
Roselawn,1,1,1,1,1,1
"Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park",17,17,17,17,17,17
"The Annex , North Midtown , Yorkville",22,22,22,22,22,22


In [28]:
print('There are {} uniques categories.'.format(len(CT_venues['Venue Category'].unique())))

There are 64 uniques categories.


In [29]:
# analyze each neighborhood

# one hot encoding
CT_onehot = pd.get_dummies(CT_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
CT_onehot['Neighbourhood'] = CT_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [CT_onehot.columns[-1]] + list(CT_onehot.columns[:-1])
CT_onehot = CT_onehot[fixed_columns]

CT_onehot.head()

,Neighbourhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Bar,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Construction & Landscaping,Cosmetics Shop,Dance Studio,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Park,Pharmacy,Pizza Place,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,Lawrence Park,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Roselawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
CT_onehot.shape

(114, 65)

In [31]:
# group by neighborhood and by taking the mean of the frequency of occurrence of each category
CT_grouped = CT_onehot.groupby('Neighbourhood').mean().reset_index()
CT_grouped

,Neighbourhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Bar,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Construction & Landscaping,Cosmetics Shop,Dance Studio,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Park,Pharmacy,Pizza Place,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.027778,0.000000,0.000000,0.000000,0.027778,0.000,0.027778,0.000000,0.00,0.055556,0.00,0.0,0.055556,0.00,0.000000,0.027778,0.000,0.083333,0.027778,0.000,0.000000,0.027778,0.00,0.000,0.000000,0.00,0.0,0.027778,0.027778,0.027778,0.055556,0.00,0.000000,0.000000,0.000,0.027778,0.055556,0.027778,0.0,0.000000,0.000000,0.00,0.000000,0.027778,0.027778,0.083333,0.000000,0.00,0.027778,0.00,0.083333,0.027778,0.00,0.00,0.000000,0.000000,0.055556,0.00,0.027778,0.027778,0.0,0.000000,0.000000,0.00
1,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.125,0.000000,0.000000,0.125,0.000000,0.000000,0.00,0.125,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.125000,0.00,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.125000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.125000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00
2,Forest Hill North & West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.20,0.000000,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.00,0.000000,0.200000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.200000,0.00,0.000000,0.000000,0.2,0.000000,0.000000,0.00
3,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.25,0.000000,0.00,0.0,0.000000,0.25,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.250000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.0,0.000000,0.000000,0.00
4,"Moore Park , Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,1.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00
5,North Toronto West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.050000,0.05,0.1,0.100000,0.00,0.000000,0.000000,0.000,0.050000,0.050000,0.000,0.000000,0.000000,0.05,0.000,0.000000,0.05,0.0,0.000000,0.000000,0.000000,0.000000,0.05,0.050000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.05,0.000000,0.050000,0.000000,0.000000,0.000000,0.05,0.050000,0.05,0.000000,0.000000,0.05,0.05,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.05
6,Roselaw

In [32]:
CT_grouped.shape

(9, 65)

In [33]:
# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in CT_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = CT_grouped[CT_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0      Sandwich Place  0.08
1        Dessert Shop  0.08
2         Pizza Place  0.08
3  Italian Restaurant  0.06
4                 Gym  0.06


----Davisville North----
              venue  freq
0             Hotel  0.12
1              Park  0.12
2  Department Store  0.12
3    Sandwich Place  0.12
4    Breakfast Spot  0.12


----Forest Hill North & West----
              venue  freq
0          Bus Line   0.2
1             Trail   0.2
2              Park   0.2
3     Jewelry Store   0.2
4  Sushi Restaurant   0.2


----Lawrence Park----
                        venue  freq
0  Construction & Landscaping  0.25
1                        Park  0.25
2                 Swim School  0.25
3                    Bus Line  0.25
4         American Restaurant  0.00


----Moore Park , Summerhill East----
                 venue  freq
0                 Park   1.0
1  American Restaurant   0.0
2            BBQ Joint   0.0
3                Hotel   0.0
4    Indian Res

In [34]:
# sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
# create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = CT_grouped['Neighbourhood']

for ind in np.arange(CT_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(CT_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape


(9, 11)

In [36]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

CT_grouped_clustering = CT_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(CT_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 4, 0, 3, 1, 2, 1, 1], dtype=int32)

In [37]:
CT_merged = CT_data

# add clustering labels
CT_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
CT_merged = CT_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

CT_merged.head() # check the last columns!
CT_merged.shape

(9, 16)

In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(CT_merged['Latitude'], CT_merged['Longitude'], CT_merged['Neighbourhood'], CT_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [39]:
# Cluster 1 
CT_merged.loc[CT_merged['Cluster Labels'] == 0, CT_merged.columns[[1] + list(range(5, CT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Central Toronto,0,Park,Trail,Sushi Restaurant,Jewelry Store,Bus Line,Farmers Market,Dessert Shop,Diner,Dog Run,Donut Shop


In [40]:
# Cluster 2
CT_merged.loc[CT_merged['Cluster Labels'] == 1, CT_merged.columns[[1] + list(range(5, CT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,1,Bus Line,Park,Construction & Landscaping,Swim School,Yoga Studio,Diner,Dog Run,Donut Shop,Farmers Market,Fast Food Restaurant
1,Central Toronto,1,Garden,Yoga Studio,Vietnamese Restaurant,Greek Restaurant,Gourmet Shop,Gas Station,Furniture / Home Store,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant
5,Central Toronto,1,Sandwich Place,Café,Coffee Shop,American Restaurant,Donut Shop,History Museum,Indian Restaurant,Liquor Store,Cosmetics Shop,Middle Eastern Restaurant
7,Central Toronto,1,Park,Yoga Studio,Gym,Gourmet Shop,Gas Station,Garden,Furniture / Home Store,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant
8,Central Toronto,1,Coffee Shop,Pub,American Restaurant,Restaurant,Fried Chicken Joint,Vietnamese Restaurant,Health & Beauty Service,Light Rail Station,Liquor Store,Pizza Place


In [41]:
# Cluster 3
CT_merged.loc[CT_merged['Cluster Labels'] == 2, CT_merged.columns[[1] + list(range(5, CT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Toronto,2,Dessert Shop,Sandwich Place,Pizza Place,Italian Restaurant,Café,Coffee Shop,Gym,Sushi Restaurant,Dance Studio,Japanese Restaurant


In [42]:
# Cluster 4
CT_merged.loc[CT_merged['Cluster Labels'] == 3, CT_merged.columns[[1] + list(range(5, CT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,3,Clothing Store,Coffee Shop,Yoga Studio,Restaurant,Café,Chinese Restaurant,Dessert Shop,Diner,Fast Food Restaurant,Furniture / Home Store


In [43]:
# Cluster 5
CT_merged.loc[CT_merged['Cluster Labels'] == 4, CT_merged.columns[[1] + list(range(5, CT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Central Toronto,4,Gym,Breakfast Spot,Park,Dog Run,Sandwich Place,Department Store,Food & Drink Shop,Hotel,Furniture / Home Store,Fried Chicken Joint


### Observation: clustering neighboorhoods in the borough of Central Toronto

By looking at each cluster, I can summarize the following foundings:

Cluster 1: The Luxury cluster. the 1st most common venues is a Jewlery, followed by ethnic restaurant.

Cluster 2: The Sports & Activities cluster. Here the 1st few most common venues in this cluster are mostly made up of children, sports or other activities related venues such as playground

Cluster 3: Spare Time cluster. Here venues to spend spare time such as Gym, Sandwich place, Park ect.

Cluster 4: Restaurant & Food cluster

Cluster 5: Shopping cluster
